<a href="https://colab.research.google.com/github/vanesalinas/HenryPI_MLOps/blob/main/funciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import gzip
import json

In [2]:
try:
    df_steam_games = pd.read_json('/content/clean_steam_games.json', lines=True)
except ValueError as e:
    print(f"Error al leer el archivo JSON: {e}")

In [3]:
df_steam_games = df_steam_games.rename(columns={'id': 'item_id'})

In [10]:
try:
    df_user_reviews = pd.read_json('/content/clean_user_reviews.json', lines=True)
except ValueError as e:
    print(f"Error al leer el archivo JSON: {e}")

In [56]:
try:
    df_user_items = pd.read_json('/content/clean_user_items.json', lines=True)
except ValueError as e:
    print(f"Error al leer el archivo JSON: {e}")

In [5]:
def developer( desarrollador : str ):
    #Transformar la columna "developer" y "price" en str en letra minuscula
    df_steam_games['developer'] = df_steam_games['developer'].astype(str).str.lower()
    df_steam_games['price'] = df_steam_games['price'].astype(str).str.lower()
    desarrollador.lower()

    #Filtrar por desarrollador
    developer_df = df_steam_games[df_steam_games['developer'].str.contains(desarrollador, case=False)]

    if developer_df.empty:
        print(f"No hay registros para el desarrollador {desarrollador}.")
        return None

    estadisticas_desarrolladores = []

    for developer, juegos in developer_df.groupby('developer'):
        total_juegos = len(juegos)
        juegos_free = juegos[juegos['price'].str.contains('free', case=False)]
        total_juegos_free = len(juegos_free)

        porcentaje_free = (total_juegos_free / total_juegos) * 100 if total_juegos > 0 else 0

        estadisticas = {
            'desarrollador': developer,
            'total_juegos': total_juegos,
            'porcentaje_free': porcentaje_free
        }

        estadisticas_desarrolladores.append(estadisticas)

    return estadisticas_desarrolladores

In [6]:
developer('nikita')

[{'desarrollador': 'korovkin nikita,filatov igor,lagunov roman,kristian shatov',
  'total_juegos': 1,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita', 'total_juegos': 2, 'porcentaje_free': 0.0},
 {'desarrollador': 'nikita "ghost_rus"',
  'total_juegos': 9,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita "ghost_rus",underanger thegachiwarrior',
  'total_juegos': 2,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita kaf', 'total_juegos': 2, 'porcentaje_free': 50.0},
 {'desarrollador': 'nikita markin,artem basko',
  'total_juegos': 1,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita nefedov',
  'total_juegos': 1,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita online',
  'total_juegos': 4,
  'porcentaje_free': 75.0}]

In [ ]:
def userdata( User_id : str ):
    '''
    Calcula cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items

    Parametros
    ----------
    User_id : str

    Retorno
    -------
    {"Usuario X" : nombre de usuario, "Dinero gastado": x USD, "% de recomendación": x%, "cantidad de items": x}

    '''

    return

In [ ]:
def UserForGenre( genero : str ):
    '''
    Devuelve el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

    Parametros
    ----------
    genero : str

    Retorno
    -------
    {"Usuario con más horas jugadas para Género X" : nombre de usuario, "Horas jugadas":[{Año: x1, Horas: n}, {Año: x2, Horas: n2}, {Año: x3, Horas: n3}]}

    '''
    genero.lower()
    df_steam_games['genres'] = df_steam_games['genres'].astype(str).str.lower()

    # Fusionar los DataFrames por item_id
    merged_df = pd.merge(df_steam_games, df_user_items, how='inner', on='item_id')

    # Filtrar por el desarrollador buscado
    genero_df = merged_df[merged_df['genres'].str.contains(genero, case=False)]

    if genero_df.empty:
        print(f"No hay registros para el desarrollador {genero}.")
        return None

    # Encontrar el usuario con más horas jugadas para el género dado
    usuario_mas_horas = genero_df.loc[genero_df['playtime_forever'].idxmax()]['user_id']

    # Calcular la acumulación de horas jugadas por año de lanzamiento
    acumulacion_horas = genero_df.groupby('release_date')['playtime_forever'].sum().reset_index()
    acumulacion_horas = acumulacion_horas.rename(columns={'release_date': 'Año', 'playtime_forever': 'Horas'})

    # Crear el diccionario de retorno
    resultado_hs = {
        f"Usuario con más horas jugadas para {genero}": usuario_mas_horas,
        "Horas jugadas": acumulacion_horas.to_dict(orient='records')
    }

    return resultado_hs

In [ ]:
UserForGenre('adventure')

In [27]:
def best_developer_year( año : int ):
    '''
    Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

    Parametros
    ----------
    año : int

    Retorno
    -------
    [{"Puesto x1" : X}, {"Puesto x2" : Y},{"Puesto x3" : Z}]

    '''
    #convertir la columna release_date en datetime
    df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')

    # Filtrar juegos para el año dado
    fecha = df_steam_games[df_steam_games['release_date'].dt.year == año]

    # Filtrar reseñas positivas y recomendadas
    recomendaciones = df_user_reviews[(df_user_reviews['sentiment_analysis'] == 2) & df_user_reviews['recommend' == True]]

    # Fusionar los DataFrames por el ID del juego
    merged_df = pd.merge(recomendaciones, fecha, how='inner', left_on='item_id', right_on='id')

    # Agrupar por desarrollador y contar las reseñas
    top_desarrolladores = merged_df.groupby('developer').size().reset_index(name='total_reseñas')

    # Ordenar en orden descendente y tomar los primeros 3
    top_desarrolladores = top_desarrolladores.sort_values(by='total_reseñas', ascending=False).head(3)

    # Crear la lista de resultados en el formato solicitado
    resultados = [{"Puesto x{}".format(i+1): desarrollador} for i, desarrollador in enumerate(top_desarrolladores['developer'])]

    return resultados

In [28]:
best_developer_year(2018)

KeyError: False

In [11]:
def developer_reviews_analysis( desarrolladora : str ):
    '''
    Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

    Parametros
    ----------
    desarrolladora : str

    Retorno
    -------
    {'desarrollador x' : [Negative = x1, Positive = x2]}

    '''
    desarrolladora.lower()
    df_steam_games['developer'] = df_steam_games['developer'].astype(str).str.lower()

    # Fusionar los DataFrames por item_id
    merged_df = pd.merge(df_steam_games, df_user_reviews, how='inner', on='item_id')

    # Filtrar por el desarrollador buscado
    desarrolladora_df = merged_df[merged_df['developer'].str.contains(desarrolladora, case=False)]

    if desarrolladora_df.empty:
        print(f"No hay registros para el desarrollador {desarrolladora}.")
        return None

    #contar la cantidad de reseñas
    for developer in desarrolladora_df.groupby('developer'):
        cantidad_positivas = (desarrolladora_df['sentiment_analysis'] == 2).sum()
        cantidad_negativas = (desarrolladora_df['sentiment_analysis'] == 0).sum()

        resultado = {
            'positivas': cantidad_positivas,
            'negativas': cantidad_negativas
        }

    return {desarrolladora: [f"{key} = {value}" for key, value in resultado.items()]}

In [12]:
developer_reviews_analysis('nikita')

{'nikita': ['positivas = 3', 'negativas = 0']}